# Twitter Sentiment Analysis

In this notebook, my aim is to get most recent bunch of tweets from twitter and clean strings, afterwards I did sentiment analysis on each tweet one by one and assign them some scores to decide positivity/negativity of it. 

The data below scrabbed at 6th of July in 2021 and top 100 tweets were filtered with only contains BTC hash to give a short example. 

Note : It is required to sign up with your own credientials as developer account to be able to wrap data through tweepy package. That is why I imported my credientials as a package called 'TwitterKeys' so I can use them to receive data through API.


In [1]:
import tweepy, json, TwitterKeys,csv,re, time                  # Python wrapper around Twitter API
import pandas as pd
import numpy as np
from datetime import date
from datetime import datetime
import matplotlib.pyplot as plt
from textblob import TextBlob as tb
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from nltk.tokenize import regexp_tokenize, TweetTokenizer,word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/burcin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(TwitterKeys.API_TOKEN, TwitterKeys.API_KEY)

auth.set_access_token(TwitterKeys.ACCESS_TOKEN, TwitterKeys.ACCESS_KEY)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


After authentication, I wrapped data through twitter api, it is optional to set a beginning date to get past tweets, however there are some drawbacks of that option. Firstly you cannot set a time together with date and more importantly, it is allowed to receive tweets at most 7 days before current date. 

Then I saved tweets as a data frame to process..

In [3]:
search_words = "#BTC"


In [4]:
new_search = search_words + " -filter:retweets" # Filtered retweets
new_search

'#BTC -filter:retweets'

In [5]:
tweets = tweepy.Cursor(api.search,
                       q=new_search,
                       lang="en"
                       #since=date_since
                      ).items(100)

df = pd.DataFrame(data = [[tweet.user.screen_name, tweet.user.location,tweet.created_at,tweet.text] for tweet in tweets],columns=['user', "location","tweet_date","tweet_text"])

In [6]:
df.shape

(100, 4)

In [7]:
df.head()

,user,location,tweet_date,tweet_text
0,VM101087,,2021-07-06 12:07:23,@JesterDefi_ @BinanceChain Good project &amp; ...
1,Edward686868,Singapore,2021-07-06 12:07:22,@PIantainChips I’ll stick with #btc thanks
2,magnapushpa,,2021-07-06 12:07:12,@keplerswap Thanks for giving us such a great ...
3,JDSecord,"Phoenix, AZ",2021-07-06 12:06:57,@garyblack00 Let’s keep in mind he passed on b...
4,coinok,,2021-07-06 12:06:49,Buy Crypto With Your Credit Card / Fast Crypto...


In [8]:
pd.options.display.max_colwidth = 2000
# Wrote a pattern that matches with all links (urls) 
pattern = r'http\S+'

# Use the pattern on the last tweet in the tweets list
print(df.tweet_text.apply(lambda x:  regexp_tokenize(x, pattern=pattern))) 
#print(df.tweet_text.apply(lambda x: re.search("(?P<url>https?://[^\s]+)", x).group("url")))


0                              [https://t.co/otdDFpo2Oq]
1                                                     []
2                              [https://t.co/Yvct3I8KHh]
3                              [https://t.co/mywBEaSbeT]
4     [https://t.co/GwaCvQyQbc, https://t.co/Sb4Pw63Lvv]
                             ...                        
95                                                    []
96                             [https://t.co/adHnhWz5VA]
97                             [https://t.co/LfPri2wZVw]
98                             [https://t.co/gheS1PMPPl]
99                             [https://t.co/cySyN8HqAg]
Name: tweet_text, Length: 100, dtype: object


Now I am going to drop urls from tweets as well as dropped tweets with same contents and generated a dictionary to set tweet texts as keys..

After that, I dropped english stopwords, hashtags and mentions too for more clean texts. 

In [9]:
tknzr = TweetTokenizer()
tweet_dict = {}




for i in range(0,len(df)):
    tweettext = tknzr.tokenize(re.sub(r'http\S+', "",df.tweet_text[i])) # dropped urls 
    tweettext = [ t for t in tweettext if t not in stopwords.words('english') and len(t)> 1 and t[0] != '#' and t[0] != '@'] # remove hastags, mentions and stopwords
    if tweet_dict.get(str(tweettext)) != 1:
        tweet_dict[str(tweettext)]=1
    #tweets = [word_tokenize(re.sub(r"""["?,$!:#@/[///\]\)\.?\+\-]|'(?!(?<! ')[ts])""", "",key)) for key in tweet_dict.keys()]
    tweets = [key for key in tweet_dict.keys() if [k.isalpha() for k in key]]
    


In [10]:
tweets

["['Good', 'project', 'Success', 'For', 'Developer', 'Join', 'guys']",
 "['stick', 'thanks']",
 "['Thanks', 'giving', 'us', 'great', 'opportunity', 'supporting', 'always', 'development', 'team', 'moon']",
 "['Let', 'keep', 'mind', 'passed', 'APPL', 'AMZN', 'getting', 'APPL', 'within', 'last', 'couple', 'year']",
 "['Buy', 'Crypto', 'With', 'Your', 'Credit', 'Card', 'Fast', 'Crypto', 'Exchange']",
 "['hope', 'win', 'Thanks', 'opportunity', 'bc1qrpzev8y32lmp0z3q68qy5anq77efjfrmudsdpn']",
 "['At', 'time', '...', 'really', 'want', 'lose', 'support', '...', 'pretty', 'nail', 'biting', 'times']",
 "['need', 'anyone', 'else', 'FUCK', 'THE', 'SYSTEM', '...', 'NO', 'ONE', 'CAN', 'STOP']",
 "['NO', 'matter', 'terrible', 'life', 'say', 'load', 'LOVE', 'MY', 'LIFE', 'whisper', 'also', 'love']",
 "['shooting', 'Use', 'find', 'COINS', 'moving', 'Track', 'crypto', 'HIGH', 'buyers']",
 "['INARI', 'FIRST', 'EVER', 'AUTOMATED', 'BUYBACK', 'CONTRACT', 'BUYBACK', 'WALLET', 'CONTAINS', '70', 'ETH', 'DEXT']

In [11]:
df.tweet_text[0] # raw version

'@JesterDefi_ @BinanceChain Good project &amp; Success For Developer \nJoin guys\n@Akash87869857\n@AnkushS28962162… https://t.co/otdDFpo2Oq'

In [12]:
tweets[0] # cleaned version

"['Good', 'project', 'Success', 'For', 'Developer', 'Join', 'guys']"

In [13]:
# In this cell I transfered ready to use trained BERT model provided from Hugging Face 

classifier = pipeline("sentiment-analysis")


In [14]:
[item["label"] for item in classifier(tweets[0])]

['POSITIVE']

In [15]:
# This fuction provided from vadersentiment library to score tweets 
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(tweets[0])

{'neg': 0.0, 'neu': 0.312, 'pos': 0.688, 'compound': 0.8316}

Last but not least I created a data frame which includes scores provided from textblob, BERT and vadersentiment libraries. 

With this clean data frame it can be evaluated by comparing results. 

In [16]:
# values from textblob
pol = []
sub = []
# values from transformers
label = []
score = []
# values from vader sentiment
neg_val = []
pos_val = []
neu_val = []
comp_val = []

for j in tweets:
    tx = tb(j)
    pol.append(tx.sentiment.polarity)
    sub.append(tx.sentiment.subjectivity)
    label.append([item["label"] for item in classifier(j)])
    score.append([item["score"] for item in classifier(j)] )  
    
    neg_val.append(analyzer.polarity_scores(j)['neg'])
    pos_val.append(analyzer.polarity_scores(j)['pos'])
    neu_val.append(analyzer.polarity_scores(j)['neu'])
    comp_val.append(analyzer.polarity_scores(j)['compound'])


In [17]:
df_pols = pd.DataFrame({"polarity":pol,"subjectivity":sub, 'label' : label, 'score': score, 'neg_val': neg_val, 'pos_val': pos_val, 'neu_val' : neu_val,'comp_val': comp_val, 'text':tweets})


In [18]:
df_pols

,polarity,subjectivity,label,score,neg_val,pos_val,neu_val,comp_val,text
0,0.500000,0.300000,[POSITIVE],[0.9996802806854248],0.0,0.688,0.312,0.8316,"['Good', 'project', 'Success', 'For', 'Developer', 'Join', 'guys']"
1,0.200000,0.200000,[POSITIVE],[0.9982790946960449],0.0,0.744,0.256,0.4404,"['stick', 'thanks']"
2,0.416667,0.400000,[POSITIVE],[0.9997115135192871],0.0,0.751,0.249,0.9337,"['Thanks', 'giving', 'us', 'great', 'opportunity', 'supporting', 'always', 'development', 'team', 'moon']"
3,0.000000,0.066667,[NEGATIVE],[0.9957703351974487],0.0,0.000,1.000,0.0000,"['Let', 'keep', 'mind', 'passed', 'APPL', 'AMZN', 'getting', 'APPL', 'within', 'last', 'couple', 'year']"
4,0.200000,0.600000,[NEGATIVE],[0.758206844329834],0.0,0.245,0.755,0.3818,"['Buy', 'Crypto', 'With', 'Your', 'Credit', 'Card', 'Fast', 'Crypto', 'Exchange']"
...,...,...,...,...,...,...,...,...,...
91,0.000000,0.000000,[NEGATIVE],[0.87502521276474],0.0,0.318,0.682,0.4215,"['btc', 'Will', 'sharing', 'prediction', 'within', 'next', '24hours']"
92,0.000000,0.000000,[POSITIVE],[0.9996468424797058],0.0,0.556,0.444,0.3612,"['Thank', 'offering', 'discount']"
93,0.000000,0.066667,[NEGATIVE],[0.9850741624832153],0.0,0.000,1.000,0.0000,"['Prices', 'update', 'USD', 'Last', 'hour', 'BTC', '34086', '+0.46', 'ETH', '2295.12', '+1.17', 'BNB', '311.54']"
94,0.000000,0.066667,[NEGATIVE],[0.9814916253089905],0.0,0.000,1.000,0.0000,"['Prices', 'update', 'EUR', 'Last', 'hour', 'BTC', '28811.5', '+0.48', 'ETH', '1938.05', '+1.02', 'BNB', '263.3']"
